In [ ]:
# Import Dependencies
import cv2
import math
import time
import serial
import urllib3
import emoji
import smtplib
import numpy as np
import pandas as pd
from datetime import datetime
from flask import Flask, render_template, request, render_template_string, jsonify
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.colors import red, blue, black
from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.parser import Parser
from email.mime.base import MIMEBase
import matplotlib.pyplot as plt
from matplotlib.pyplot import style
style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
# Mimic Inital BPM
initial_bpm = 80
start_pressed = False
i = 1
bpm = 0
data_analytics_file_name = ''
doctor_email = 'xxxxxxxxxx@gmail.com'
doctor_passwrd = 'xxxxxxxx'

In [ ]:
# Open Serial COM Port
Arduino_serial = serial.Serial("COM4", 9600)

In [ ]:
# *************************************************
# ***************** FLASK APP *********************
# *************************************************
app = Flask(__name__, template_folder='templates')


# **********************
# App route to Main Page
# **********************
@app.route('/')
def index():
    return render_template('main.html', beats_per_minute= initial_bpm)


# *****************************************
# Function to get User Name, Gender and Age
# *****************************************
def name_and_gender(usrName=None):
    usr_name = usrName.split(',')[0]
    usr_gender = usrName.split(',')[1]
    usr_age = usrName.split(',')[2]
    usr_email = usrName.split(',')[3]
    print('\nPatient Name: {0}\t Patient Gender: {1}\t Patient Age: {2}\t Patient e-mail: {3}\n'.format(usr_name, usr_gender, usr_age, usr_email))
    with open('user_name.txt','w') as f:
        f.write(usrName)
        

# ********************************
# Function to Plot Heart Rate Data
# ********************************
def plotHeartBeat(file=None):
    plt.figure(figsize=(40,20))
    
    try:
        # Read Data from file and Show the Plot
        #hrt_data = numpy.loadtxt(file)
        hrt_data = np.genfromtxt(file, dtype=int) 
        print(hrt_data, type(hrt_data))
        name = file.split('.')[0] + '_heartRate.png'
        if (len(hrt_data) > 1):
            plt.plot(hrt_data, label='hrt_data', color="blue")
            plt.legend()
            plt.savefig(name)
            plt.show
    except:
        print('Need more data...')
        

# ********************************************************
# Function to Write data to CSV file and Plot in Real Time
# ********************************************************
def save_and_plot(fileName=None):
    threshold = 550
    x = 1
    usr_heart_rate = []
    usr_data_timestamp = []
    
    with open(fileName,'w') as f:
        while(x < 3000):
            x += 1
            ser_data = Arduino_serial.readline().strip()
            if (int(ser_data.decode()) > threshold):
                #print(ser_data.decode())
                usr_heart_rate.append(int(ser_data.decode()))
                usr_data_timestamp.append(str(datetime.time(datetime.now()))[:-7])
                f.write(ser_data.decode())
                f.write('\n')
    # Once all data is collected, plot it
    plotHeartBeat(file=fileName)
    
    name = fileName.split('.')[0]
    # Convert Array of values to dataframe
    df = pd.DataFrame({'Heart_Rate':np.asarray(usr_heart_rate), 'Time':np.asarray(usr_data_timestamp)})
    df.to_csv(name+'_analytics_data.csv', index=False)
    data_analytics_file_name = name+'_analytics_data.csv'
    print('\nSaved user data as {}.csv.\n'.format(name+'_analytics_data'))
                

# ******************************
# Function to get data Analytics
# ******************************
def get_data_analytics(userDataFile=None):
    print('\nGenerating Data Analytics...\n')
    name = userDataFile.split('_')[0]
    user_name = userDataFile[:-19]
    
    df = pd.read_csv(userDataFile)
    #Calculate moving average with 0.75s in both directions, then append do dataset
    #One-sided window size, as proportion of the sampling frequency
    hrw = 0.75
    fs = 100
    
    # Calculate moving average
    mov_avg = df['Heart_Rate'].rolling(int(hrw*fs)).mean()
    # Impute where moving average function returns NaN, which is the beginning of the signal where x hrw
    avg_hr = (np.mean(df.Heart_Rate))
    mov_avg = [avg_hr if math.isnan(x) else x for x in mov_avg]
    # For now we raise the average by 20% to prevent the secondary heart contraction from interferce
    mov_avg = [x*1.2 for x in mov_avg]
    # Append the moving average to the dataframe
    df['Heart_Rate_Rollingmean'] = mov_avg
    
    #Mark regions of interest
    window = []
    peaklist = []
    listpos = 0 #We use a counter to move over the different data columns
    for datapoint in df.Heart_Rate:
        # Get local mean
        rollingmean = df.Heart_Rate_Rollingmean[listpos]
        # If no detectable R-complex activity -> do nothing
        if (datapoint < rollingmean) and (len(window) < 1):
            listpos += 1
            # If signal comes above local mean, mark ROI
        elif (datapoint > rollingmean):
            window.append(datapoint)
            listpos += 1
        else: # If signal drops below local mean -> determine highest point
            maximum = max(window)
            # Anotate the position of the point on the X-axis
            beatposition = listpos - len(window) + (window.index(max(window)))
            # Add detected peak to list
            peaklist.append(beatposition)
            # Clear marked ROI
            window = []
            listpos += 1
    
    ybeat = [df.Heart_Rate[x] for x in peaklist] #Get the y-value of all peaks for plotting purposes
    plt.figure(figsize=(40,20))
    plt.title("Detected Peaks in Signal and Moving Averages")
    plt.xlim(0,2500)
    plt.plot(df.Heart_Rate, alpha=0.5, color='blue', label='Heart_Rate') #Plot semi-transparent HR
    plt.plot(mov_avg, color ='green', label='Moving_avg') #Plot moving average
    plt.scatter(peaklist, ybeat, color='red', label='Peaks') #Plot detected peaks
    plt.legend(loc='best')
    plt.savefig('user_analysis.png')
    plt.show()
    
    print('\nCalculating Average Heart Rate...\n')
    RR_list = []
    cnt = 0
    while (cnt < (len(peaklist)-1)):
        RR_interval = (peaklist[cnt+1] - peaklist[cnt]) #Calculate distance between beats in # of samples
        ms_dist = ((RR_interval / fs) * 1000.0) #Convert sample distances to ms distances
        RR_list.append(ms_dist) #Append to list
        cnt += 1
    
    # 60000 ms (1 minute) / average R-R interval of signal
    bpm = 60000 / np.mean(RR_list)
    print("\nAverage Heart Beat is: %.01f BPM\n" %bpm)
    plt.figure(figsize=(40,20))
    plt.title("Detected Peaks in Signal")
    plt.xlim(0,2500)
    plt.plot(df.Heart_Rate, alpha=0.5, color='blue', label="raw signal")
    plt.plot(mov_avg, color ='green', label="moving average")
    plt.scatter(peaklist, ybeat, color='red', label="average: %.1f BPM" %bpm)
    plt.legend(loc=4, framealpha=0.6)
    plt.savefig(user_name+'_avg_heartRate.png')
    plt.show()
    return bpm


# ***********************************************
# ***************** STEP-3 **********************
# ***********************************************

# *************************************************
# Function to Capture User Image for Medical Report
# *************************************************
def capture_image(user_name=None):
    cam = cv2.VideoCapture(0)
    while True:
        ret, frame = cam.read()
        cv2.imshow("CAPTURED IMAGE", frame)

        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape Hit, Closing Session...")
            break

        elif k%256 == 32:
            # SPACE pressed
            img_name = user_name+'.png'
            cv2.imwrite(img_name, frame)
            print("\n{} written !!\n".format(img_name))

    cam.release()
    cv2.destroyAllWindows()
    return img_name


# Function to Pre-Process the Image (Crop Image)
def preprocessImage(image=None):
    img = cv2.imread(image)
    crop_img = img[145:1297, 180:2740]
    cv2.imwrite(image, crop_img)

# Function to Generate and Save Health Report as PDF File
def save_report(user_name=None, user_gender=None, user_age=None, report_type = 'Cardiology', avg_bpm=None, user_img_path=None):
    print('\nGenerating Medical Report...\n')
    c = canvas.Canvas(user_name+'.pdf', pagesize=letter)
    c.drawInlineImage('./logo3.jpg', 50,725, width=70,height=50)
    c.drawInlineImage('./logo2.png', 125,725, width=50,height=50)

    c.setFont('Helvetica-Bold', 16)
    c.setFillColor(red)
    c.drawString(250,750,'MEDICAL REPORT')
    c.setLineWidth(1)
    c.line(250,747,390,747)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(480,750,'DATE: ')
    c.setFont('Helvetica', 10)
    c.drawCentredString(550,750,str(datetime.date(datetime.now())))

    c.drawInlineImage(user_img_path, 450,580, width=130,height=120)
    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,670,'Patient''s Name: ')
    c.setFont('Helvetica', 11)
    c.setFillColor(blue)
    c.drawString(240,672,user_name.upper())
    c.setLineWidth(0.3)
    c.line(150,670,400,670)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,650,'Patient''s Gender: ')
    c.setFont('Helvetica', 11)
    c.setFillColor(blue)
    if (user_gender == 'M'):
        c.drawString(250,652,'MALE')
    elif (user_gender == 'F'):
        c.drawString(250,652,'FEMALE')
    c.setLineWidth(0.3)
    c.line(150,650,400,650)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,630,'Patient''s Age: ')
    c.setFont('Helvetica', 11)
    c.setFillColor(blue)
    c.drawString(255,632,str(user_age))
    c.setLineWidth(0.3)
    c.line(150,630,400,630)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,610,'Report Type: ')
    c.setFont('Helvetica', 11)
    c.setFillColor(blue)
    c.drawString(235,612,report_type.upper())
    c.setLineWidth(0.3)
    c.line(150,610,400,610)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,590,'Consultant: ')
    c.setFont('Helvetica', 11)
    c.setFillColor(blue)
    c.drawString(240,592,'AASHI DUTT')
    c.setLineWidth(0.3)
    c.line(150,590,400,590)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,558,'HEART ECG')
    c.setLineWidth(0.3)
    c.line(50,550,590,550)
    preprocessImage(image='./'+user_name+'_heartRate.png')
    c.drawInlineImage('./'+user_name+'_heartRate.png', 50,340, width=530,height=200)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,307,'HEART ECG MOVING AVERAGES & AVERAGE BPM')
    c.setLineWidth(0.3)
    c.line(50,305,580,305)
    preprocessImage(image='./'+user_name+'_avg_heartRate.png')
    c.drawInlineImage('./'+user_name+'_avg_heartRate.png', 50,93, width=530,height=200)

    c.setFont('Helvetica-Bold', 11)
    c.setFillColor(black)
    c.drawString(50,70,'REPORT SUMMARY')
    c.setLineWidth(0.3)
    c.line(50,67,580,67)
    c.setFont('Helvetica', 10)
    c.setFillColor(black)
    c.drawString(50,52,'In this test, the Heart Beat readings of {} were collected for a time span of 3 to 4 minutes.'.format(user_name))
    c.drawString(50,38,'During the test, the patient''s heart showed an Average Beats per Minute of {} BPM.'.format(round(avg_bpm),2))
    c.drawString(50,24,'The ECG data for the test with details is included above. Based on patient''s data, the patient''s heart is healthy.')
    c.save()


    
# Check if Internet is Available or not by Pinging on Google Server
def isInternetAvailable():
    http = urllib3.PoolManager()
    url = 'http://216.58.192.142'
    # Try to Ping the Google.com server, if Internet available, return True
    try:
        response = http.request('GET', url)
        return True
    # else return False
    except urllib3.connection.ConnectionError as e: 
        return False

    
# Send PDF Report to Patient via Email
def send_email(sender_email=None, reciever_email=None, file_path=None, auth_usr_name=None, auth_usr_passwrd=None):
    print('\nSending Health Report to User via e-mail...\n')
    msg = MIMEMultipart()
    msg['Subject'] = 'Latest Health Report'
    msg['From'] = sender_email
    msg['To'] = reciever_email

    msg_body = emoji.emojize('Hi,\n\n Attached, please find your latest Health Report. :file_folder: :page_with_curl:\n\nBest,\nYour Health Consultant')
    body = MIMEText(msg_body)
    msg.attach(body)

    part = MIMEBase('application', "octet-stream")
    part.set_payload(open(file_path, "rb").read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment; filename={}'.format(file_path))
    msg.attach(part)

    # Check if Internet is Available, then try sending email
    if (isInternetAvailable()):
        try:
            mailer = smtplib.SMTP('smtp.gmail.com',587)
            mailer.ehlo()
            mailer.starttls()
            mailer.ehlo()
            mailer.login(user=auth_usr_name, password=auth_usr_passwrd)
            mailer.sendmail(sender_email, reciever_email, msg.as_string())
            mailer.close()
            print(emoji.emojize('Your email has been delivered sucessfully !! :sunglasses:'))
        except error as e:
            print('ERROR: ',e)
    else:
        print(emoji.emojize('\nInternet Not Available !! :cry: Please try again later...\n'))
    
    
        
@app.route('/', methods=['GET','POST'])
def data():
    # Variable to hold BPM Values
    bpm = 0
    text = ' '
    ser_start = False
    usr_name = ''
    
    # Format ( request.form[name] == value )
    if (request.method == 'POST'):
        
        # Get User Name and Gender for Generating Report
        if (request.form['button'] == 'user_name_submit'):
            user_name = request.form['user_name']
            name_and_gender(usrName=user_name)
        
        # Start Recording Data in CSV File and Plot Data in Real Time
        elif (request.form['button'] == 'start'):
            with open('user_name.txt','r') as f:
                text = f.read()
            name = text.split(',')[0]
            # Function to Start Reading the data
            save_and_plot(fileName=name+'.csv')
        
        elif (request.form['button'] == 'stop'):
            #Arduino_serial.close()
            with open('user_name.txt','r') as f:
                text = f.read()
            name = text.split(',')[0]
            gender = text.split(',')[1]
            age = text.split(',')[2]
            patient_email = str(text.split(',')[3])
            file_name = name+'_analytics_data.csv'
            # Get User Image
            img_name = capture_image(user_name=name)
            img_path = './'+img_name
            print('\nImage Saved to Path: \n',img_path)
            # Get Average BPM
            bpm = get_data_analytics(userDataFile=file_name)
            # Save Report as PDF
            save_report(user_name=name, user_gender=gender, user_age=age, report_type = 'Cardiology', avg_bpm=bpm, user_img_path=img_path)
            print('\nHealth Report Saved for {}\n'.format(name))
            send_email(sender_email=doctor_email, reciever_email=patient_email, file_path=name+'.pdf', auth_usr_name=doctor_email, auth_usr_passwrd=doctor_passwrd)
            return render_template('main.html', beats_per_minute= round(bpm,2))
        
        elif (request.form['button'] == 'showPlot'):
            ser_data = Arduino_serial.readline().strip()
            
            if (ser_data > threshold):
                user_heart_beat_data.append(ser_data.decode())
    
        
    elif (request.method == 'GET'):
        return render_template('main.html',  beats_per_minute= bpm)
    
    return render_template('main.html', beats_per_minute= initial_bpm)


# Main Function
if __name__ == '__main__':
    app.run()